In [3]:
import logging
# log_level = logging.INFO
# log_level = logging.DEBUG
# log_level = 15
log_level = 25 # ANALYSIS
logging.basicConfig(level=log_level)
logger = logging.getLogger(__name__)
logger.setLevel(log_level)


In [4]:
import os
import pandas as pd
from openai import OpenAI
from regulations_rag.rerank import RerankAlgos
from cemad_rag.cemad_chat import CEMADChat


In [13]:
openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)
key = os.getenv('excon_encryption_key')

#rerank_algo  = RerankAlgos.MOST_COMMON
rerank_algo  = RerankAlgos.LLM
chat = CEMADChat(openai_client = openai_client, 
                    decryption_key = key, 
                    rerank_algo = rerank_algo, 
                    user_name_for_logging = 'test_user')



In [6]:
stand_alone_questions = []

stand_alone_questions.append('I am part of a group of companies and I want to make a service payment to one of our subsidiary companies. Can I proceed?') # I get no answer here
stand_alone_questions.append('Can I make a fee payment to a non-resident related party') # version of the last question which uses the correct wording
stand_alone_questions.append('What is merchanting trade?')
stand_alone_questions.append('Tell me about the time lag between paying funds away to the foreign supplier and receiving funds from the foreign importer')
stand_alone_questions.append('Can a customer from South Africa purchase goods from China and supply the goods to the USA?')
stand_alone_questions.append('Can a customer from South Africa import goods from another country but supply the goods to a different country?')
stand_alone_questions.append('I want to make an import where a payment will only be done over 12 months. Can this be done?')

stand_alone_questions.append('what is the current date of manual?')
stand_alone_questions.append('can a South African trust pay funds abroad?')
stand_alone_questions.append('how much can an individual invest offshore?')
stand_alone_questions.append('how much can a South African entity invest abroad')
stand_alone_questions.append('how does a non-resident evidence shareholding in a SA company?')
stand_alone_questions.append('what is a forex broker') # unable to answer
stand_alone_questions.append('what is a treasury outsourcer')
stand_alone_questions.append('how do I apply for Imports Undertaking?')
stand_alone_questions.append('Imports dispensation')
stand_alone_questions.append('What is required for amd imports undertaking dispensation?')


In [8]:
question_answer_pairs = []

for question in stand_alone_questions:
    #question = stand_alone_questions[0]
    chat.reset_conversation_history()
    response = chat.chat_completion(question)
    question_answer_pairs.append([question, response])


ANALYSIS:regulations_rag.regulation_chat:user to test_user: I am part of a group of companies and I want to make a service payment to one of our subsidiary companies. Can I proceed?
ANALYSIS:regulations_rag.regulation_chat:assistant to test_user: I was unable to find any relevant documentation to assist in answering the question. Can you try rephrasing the question?
ANALYSIS:regulations_rag.regulation_chat:user to test_user: Can I make a fee payment to a non-resident related party
ANALYSIS:regulations_rag.regulation_chat:assistant to test_user: Yes, you can make a fee payment to a non-resident related party, but such requests should be submitted to the Financial Surveillance Department for consideration. 

Reference: B.3(C)(ii),
ANALYSIS:regulations_rag.regulation_chat:user to test_user: What is merchanting trade?
ANALYSIS:regulations_rag.regulation_chat:assistant to test_user: Merchanting trade refers to transactions by residents where goods are purchased from a foreign supplier and s

In [10]:
import pandas as pd
df = pd.DataFrame(question_answer_pairs, columns = ["Question", "Answer"])
df.to_csv("./working/regression.csv", encoding="utf-8", sep = "|")

In [11]:
followup_questions = []
followup_questions.append(['I am working on contract in South Africa. I am a foreign national. Can I send money home?','What documentary evidence would I need?']) 
followup_questions.append(['I have a company non resident foreign currency account and I would like to pay salaries of my employees cross border. Can this be done?', 'Is there any documentation required?'])


In [14]:
followup_question_answers = []

for question_with_followup in followup_questions:
    chat.reset_conversation_history()
    question = question_with_followup[0]
    response = chat.chat_completion(question)
    followup_question = question_with_followup[1]
    followup_response = chat.chat_completion(followup_question)

    question_answer_pairs.append([question, response, followup_question, followup_response])


ANALYSIS:regulations_rag.regulation_chat:user to test_user: I am working on contract in South Africa. I am a foreign national. Can I send money home?
ANALYSIS:regulations_rag.regulation_chat:assistant to test_user: Yes, as a foreign national working on contract in South Africa, you can send money home. According to the South African Exchange Control Manual, Authorised Dealers may allow the transfer of funds abroad in respect of employment contracts involving non-residents who are employed in South Africa, provided that the payments are commensurate with the work undertaken. Additionally, as a foreign national temporarily in South Africa, you are allowed to transfer abroad funds accumulated during your stay, provided you can substantiate the source of these funds and the value of such funds is reasonable in relation to your income-generating activities in South Africa during the period. You are required to declare any foreign assets upon arrival and can conduct your banking on a residen

In [15]:
import pandas as pd
df = pd.DataFrame(question_answer_pairs, columns = ["Question", "Answer","Followup Question","Followup Response"])
df.to_csv("./working/regression_followup.csv", encoding="utf-8", sep = "|")

## CFC Accounts
We want to answer the question:Is there documentation required for non resident foreign currency account?
We know that a non-residend can open a CFC account E.(C)
We also know that a a CFC account is subject to certain conditions that include that the account must be part of the bank's nostro account administration, and the client cannot operate it without the approval of the Authorised Dealer. Clients are allowed to retain funds in their CFC accounts without the need to convert them into Rand, and only foreign exchange transactions that qualify as permissible credits and/or debits may be passed over these accounts. All transactions must comply with the provisions of the Authorised Dealer Manual or specific authorities from the Financial Surveillance Department. E.(B)(ii)(a)
This means I need to do an exercise to see to check "Documents" in general and also the conditions associated with nostro account administration to create an answer

In [25]:
cfc_questions = []
cfc_questions.append('Is there documentation required for non resident foreign currency account?')